## Load data

In [1]:
from collections import Counter
import pandas as pd
import numpy as np
import os
import re

#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
# nltk.download('stopwords')
remove_these = set(stopwords.words('english'))

In [2]:
REMOVE_STOPWORDS = True

In [3]:
%%time

with open('../../data/mxm_dataset_train.txt','r') as f:
    lines = f.readlines()
    words = lines[17].replace('%','').split(',')
    all_songs_dict = dict()
    for i,l in list(enumerate(lines))[18:]:
        song_info = l.split(',')
        MSDID = song_info[0]
        song_bow = [x.split(':') for x in song_info[2:]]
        song_dict = {}
        for word, word_count in song_bow:
            song_dict[int(word)] = int(word_count.replace('\n',''))
        word_lists = [[words[word-1]]*song_dict[word] for word in song_dict.keys()]
        song = [word for word_list in word_lists for word in word_list]
        if REMOVE_STOPWORDS:
            song = [w for w in song if w not in remove_these]
        all_songs_dict[str(MSDID)] = ' '.join(song).replace('\n','')

CPU times: user 24.3 s, sys: 294 ms, total: 24.6 s
Wall time: 24.8 s


In [4]:
print(len(all_songs_dict.keys()))
song_msd_ids = list(all_songs_dict.keys())

210519


In [5]:
print(np.random.choice(song_msd_ids))

TRREYEX128F42807EB


In [6]:
%%time

all_song_meta_dict = dict()
with open('../../data/mxm_779k_matches.txt','r') as f:
    lines = f.readlines()
    for i in range(18, len(lines)):
        line = lines[i].split('<SEP>')
        MSDID = line[0]
        artist = line[1]
        title = line[2]
        all_song_meta_dict[str(MSDID)] = {'artist': artist, 'title': title}

CPU times: user 1.39 s, sys: 254 ms, total: 1.64 s
Wall time: 1.67 s


In [7]:
len(all_song_meta_dict.keys())

779056

In [8]:
# all_artists = sorted(set([all_song_meta_dict[msdId]['artist'].lower() for msdId in all_song_meta_dict]))

In [9]:
d = {
    'MSDID': list(all_songs_dict.keys()),
    'cleaned_text': [all_songs_dict[x] for x in all_songs_dict.keys()]
    }
msdid_df = pd.DataFrame.from_dict(d)
print(msdid_df.shape)
msdid_df.head()

(210519, 2)


,MSDID,cleaned_text
0,TRAAAAV128F421A322,like like de got would seem someon understand ...
1,TRAAABD128F429CF47,know know know know know time time time la la ...
2,TRAAAED128E0783FAB,love love love love love love love love love l...
3,TRAAAEF128F4273421,know got got got feel let would would would ey...
4,TRAAAEW128F42930C0,like take would wo someth stay burn burn burn ...


In [10]:
d = {
    'MSDID': msdid_df['MSDID'],
    'artist': [all_song_meta_dict[x]['artist'] for x in msdid_df['MSDID']],
    'title': [all_song_meta_dict[x]['title'] for x in msdid_df['MSDID']]
    }
meta_df = pd.DataFrame.from_dict(d)
print(meta_df.shape)
meta_df.head()

(210519, 3)


,MSDID,artist,title
0,TRAAAAV128F421A322,Western Addiction,A Poor Recipe For Civic Cohesion
1,TRAAABD128F429CF47,The Box Tops,Soul Deep
2,TRAAAED128E0783FAB,Jamie Cullum,It's About Time
3,TRAAAEF128F4273421,Adam Ant,Something Girls
4,TRAAAEW128F42930C0,Broken Spindles,Burn My Body (Album Version)


In [11]:
full_df_ = pd.merge(msdid_df, meta_df, on='MSDID', how='left')
print(full_df_.shape)
full_df_.head()

(210519, 4)


,MSDID,cleaned_text,artist,title
0,TRAAAAV128F421A322,like like de got would seem someon understand ...,Western Addiction,A Poor Recipe For Civic Cohesion
1,TRAAABD128F429CF47,know know know know know time time time la la ...,The Box Tops,Soul Deep
2,TRAAAED128E0783FAB,love love love love love love love love love l...,Jamie Cullum,It's About Time
3,TRAAAEF128F4273421,know got got got feel let would would would ey...,Adam Ant,Something Girls
4,TRAAAEW128F42930C0,like take would wo someth stay burn burn burn ...,Broken Spindles,Burn My Body (Album Version)


In [54]:
my_artists_ = ['Green Day','Blink-182','Sum 41','Rise Against',
              'Four Year Strong','Three Days Grace',
              'Tame Impala','Snoop Dogg','Thin Lizzy','The Strokes',
              'Nirvana','All Time Low', 'Foo Fighters','Lil Wayne',
              'Eminem','Tom Petty','Seether', 'Shinedown',
              'Red Hot Chili Peppers','The Who', 'The Doors',
              'Linkin Park','Arctic Monkeys','The Black Keys',
              'Led Zeppelin','The Kinks','Collective Soul',
              'Avenged Sevenfold','George Harrison','Eric Clapton',
              'The Donnas','System of a Down', 'MxPx',
              'Weezer','Rancid','Atmosphere','Santigold',
              'Iron Maiden','Fall Out Boy', 'Hank Williams']

my_artists = ['The Smashing Pumpkins', 'Bruce Springsteen','Beastie Boys']

sampled_artists = list(np.random.choice(full_df_['artist'], size=500, replace=False))
# sampled_artists

In [85]:
# top_artists = full_df_['artist'].value_counts()[:10].reset_index()['index'].values
# full_df = full_df_[full_df_['artist'].isin(my_artists)].reset_index(drop=True)
# full_df = full_df_[full_df_['artist'].isin(sampled_artists)].reset_index(drop=True)
# full_df = full_df_[full_df_['artist'].isin(sampled_artists + my_artists + my_artists_)].reset_index(drop=True)
full_df = full_df_.copy()
print(full_df.shape)
full_df.head()

(210519, 4)


,MSDID,cleaned_text,artist,title
0,TRAAAAV128F421A322,like like de got would seem someon understand ...,Western Addiction,A Poor Recipe For Civic Cohesion
1,TRAAABD128F429CF47,know know know know know time time time la la ...,The Box Tops,Soul Deep
2,TRAAAED128E0783FAB,love love love love love love love love love l...,Jamie Cullum,It's About Time
3,TRAAAEF128F4273421,know got got got feel let would would would ey...,Adam Ant,Something Girls
4,TRAAAEW128F42930C0,like take would wo someth stay burn burn burn ...,Broken Spindles,Burn My Body (Album Version)


## Train model and get embeddings

In [86]:
%%time

# get training data
data = full_df['cleaned_text'].tolist()
# test_data = test_df['cleaned_text'].tolist()
train_corpus = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in list(enumerate(data))]
# build model
model = Doc2Vec(vector_size=50, min_count=1, epochs=10, dm=0)
model.build_vocab(train_corpus)
# train model
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 7min 41s, sys: 16.6 s, total: 7min 58s
Wall time: 5min 7s


In [87]:
model.save('trained_model_example')

In [88]:
model = Doc2Vec.load('trained_model_example')

In [89]:
emb_df = pd.DataFrame([model.docvecs[f'{i}'] for i in range(len(full_df))])
fe_df = full_df.copy()
for c in emb_df.columns:
    fe_df[c] = emb_df[c]
# fe_df[fe_df['artist'].isin(my_artists)].groupby('artist').mean().reset_index()

In [90]:
fe_df.head()

,MSDID,cleaned_text,artist,title,0,1,2,3,4,5,...,40,41,42,43,44,45,46,47,48,49
0,TRAAAAV128F421A322,like like de got would seem someon understand ...,Western Addiction,A Poor Recipe For Civic Cohesion,0.401697,0.526880,0.000193,-0.207701,0.087055,-0.078098,...,-0.441855,-0.378998,-0.175866,0.203068,-0.027629,0.477828,-0.254180,-0.115484,0.236513,0.109752
1,TRAAABD128F429CF47,know know know know know time time time la la ...,The Box Tops,Soul Deep,0.087861,-0.038411,-0.197510,-0.030250,-0.019774,-0.743675,...,-0.274182,0.128060,0.208673,0.077781,0.185496,0.068143,-0.626044,-0.538793,-0.053541,-0.068593
2,TRAAAED128E0783FAB,love love love love love love love love love l...,Jamie Cullum,It's About Time,0.212101,0.353612,-0.462079,-0.311941,0.261468,-0.583442,...,-0.165262,-0.897217,-0.154583,0.144875,-0.070821,-0.462086,-0.377535,-0.233483,0.211760,-0.161529
3,TRAAAEF128F4273421,know got got got feel let would would would ey...,Adam Ant,Something Girls,0.006594,0.402120,-0.435748,-0.142244,0.086601,-0.458696,...,-0.006869,-0.286528,-0.198836,0.117229,-0.070014,-0.559188,-0.138114,-0.108960,-0.379482,0.152984
4,TRAAAEW128F42930C0,like take would wo someth stay burn burn burn ...,Broken Spindles,Burn My Body (Album Version),0.256399,0.174773,-0.139055,0.095113,-0.103092,-0.408004,...,-0.366267,-0.334991,-0.356486,0.077660,0.222815,0.359386,-0.492463,0.079446,0.100171,-0.393555


## Visualization with t-SNE

In [91]:
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.pipeline import Pipeline

In [92]:
n = 3

pipeline = Pipeline([('scaling', StandardScaler()), ('tsne', TSNE(n_components=n, perplexity=30, random_state=42))])
song_embedding = pipeline.fit_transform(emb_df)
# tsne = TSNE(n_components=n, perplexity=30, random_state=42)
# song_embedding = tsne.fit_transform(song_df)
tsne_df = pd.DataFrame(data = song_embedding, columns = list(range(n)))
tsne_df['title'] = full_df['title']
tsne_df['artist'] = full_df['artist']

In [93]:
tsne_df.to_csv('tsne_3_componets.csv', index=False)

In [94]:
# song_df = ?

In [244]:
my_artists = ['Blink-182',
              'Tame Impala','Snoop Dogg','The Strokes',
              'Eminem',
              'Red Hot Chili Peppers','The Who',
              'Linkin Park',
              'Avenged Sevenfold','Eric Clapton']
# my_artists = tsne_df['artist'].sample(100)
# my_artists = ['Avenged Sevenfold', 'Eminem','The Strokes']
# my_artists = ['Avenged Sevenfold', 'Bob Dylan', 'Hank Williams','Dolly Parton']
my_artists

['Blink-182',
 'Tame Impala',
 'Snoop Dogg',
 'The Strokes',
 'Eminem',
 'Red Hot Chili Peppers',
 'The Who',
 'Linkin Park',
 'Avenged Sevenfold',
 'Eric Clapton']

In [245]:
tsne_df.artist.value_counts()[:30]

Carly Simon              136
Bruce Springsteen        131
Frank Sinatra            117
Johnny Cash              115
The Cure                 113
Neil Diamond             112
Joan Baez                111
Less Than Jake           111
The Hollies              110
Elvis Presley            110
Bob Dylan                110
James Taylor             109
Willie Nelson            108
Aerosmith                107
Beastie Boys             106
Céline Dion              104
Randy Travis             103
Neil Young               103
Dolly Parton             102
MxPx                     101
The Smashing Pumpkins    101
David Bowie              101
Billie Holiday           100
Cat Stevens               99
NOFX                      98
Lou Reed                  98
Bryan Ferry               98
Cradle Of Filth           98
ZZ Top                    97
Intocable                 97
Name: artist, dtype: int64

In [246]:
plot_df = fe_df[fe_df['artist'].isin(my_artists)].reset_index(drop=True)

In [247]:
emb_df_ = emb_df[fe_df['artist'].isin(my_artists)]

In [248]:
print(tsne_df.shape)
tsne_df.head()

(210519, 5)


,0,1,2,title,artist
0,15.544641,-7.086922,0.319564,A Poor Recipe For Civic Cohesion,Western Addiction
1,-5.121363,-9.543425,-9.229302,Soul Deep,The Box Tops
2,-4.169483,-12.589064,-2.418897,It's About Time,Jamie Cullum
3,1.856107,7.004950,-2.476534,Something Girls,Adam Ant
4,6.806807,-1.747844,6.527610,Burn My Body (Album Version),Broken Spindles


In [249]:
# tsne_df = tsne_df.groupby('artist').mean().reset_index()

In [250]:
tsne_df_ = tsne_df[fe_df['artist'].isin(my_artists)].reset_index(drop=True)
tsne_df_['title'] = plot_df['title']
tsne_df_['artist'] = plot_df['artist']
tsne_df_.columns = ['t-SNE dimension 0','t-SNE dimension 1','t-SNE dimension 2','title','artist']
print(tsne_df_.shape)
tsne_df_.head()

(525, 5)


,t-SNE dimension 0,t-SNE dimension 1,t-SNE dimension 2,title,artist
0,6.755438,12.022617,-4.888264,The One And Only (Edited),Snoop Dogg
1,2.356681,-11.784923,-2.789675,Not Now,Blink-182
2,3.059399,-1.241729,6.286456,A New Hope,Blink-182
3,6.596621,14.061868,-5.131838,Get On Top (Album Version),Red Hot Chili Peppers
4,7.413083,4.659088,9.663710,Naked In The Rain (Album Version),Red Hot Chili Peppers


In [251]:
tsne_df_

,t-SNE dimension 0,t-SNE dimension 1,t-SNE dimension 2,title,artist
0,6.755438,12.022617,-4.888264,The One And Only (Edited),Snoop Dogg
1,2.356681,-11.784923,-2.789675,Not Now,Blink-182
2,3.059399,-1.241729,6.286456,A New Hope,Blink-182
3,6.596621,14.061868,-5.131838,Get On Top (Album Version),Red Hot Chili Peppers
4,7.413083,4.659088,9.663710,Naked In The Rain (Album Version),Red Hot Chili Peppers
...,...,...,...,...,...
520,1.453490,-8.320243,-0.390243,Unbound [The Wild Ride] (Album Version),Avenged Sevenfold
521,-3.411914,-15.171261,10.287209,Shadow Of The Day (Album Version),Linkin Park
522,8.634074,-1.029791,-2.887656,Thick And Thin,Avenged Sevenfold
523,4.788575,10.448133,-4.437519,No Apologies,Eminem


In [252]:
import plotly.express as px

for x in [('t-SNE dimension 0','t-SNE dimension 1'),
          ('t-SNE dimension 0','t-SNE dimension 2'),
          ('t-SNE dimension 1','t-SNE dimension 2')]:
    fig = px.scatter(tsne_df_, 
                     x=x[0],
                     y=x[1],
                     hover_data=['artist','title'],
                     color='artist',
                     opacity=.8,
                     template='simple_white')\
            .update_traces(marker=dict(size=15))\
            .update_layout(
                title={
                    'text': "Song Lyric Embeddings",
                    'y':0.95,
                    'x':0.19,
                    'xanchor': 'center',
                    'yanchor': 'top'})
    fig.show()

In [253]:
for x in [('t-SNE dimension 0','t-SNE dimension 1','t-SNE dimension 2')]:   
    fig = px.scatter_3d(tsne_df_, 
                     x=x[0],
                     y=x[1],
                     z=x[2],
                     hover_data=['artist', 'title'],
                     color='artist',
                     template='simple_white')
    fig.show()